In [1]:
import os
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
import seaborn as sns

from scipy.stats import pearsonr

sns.set_style("darkgrid")
np.random.seed(930525)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 200)

warnings.simplefilter('once')

%matplotlib inline
%load_ext watermark
%watermark --iversions

numpy   1.19.5
pandas  1.1.4
seaborn 0.10.1



In [3]:
import plot_utils as pu

/home/bhillmann/.conda/envs/type_1/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
from glob import glob

import joblib

classifiers = []

for file in glob("../data/clf.sklearn.*"):
    clf = joblib.load(file)
    classifiers.append(clf)

In [13]:
X

,hits,mean_coverage,sd_coverage,percent_binned_coverage,mean_binned_coverage,sd_binned_coverage,expected_percent_coverage,shannon_entropy,percent_max_uncovered_region,largest_pileup,...,gf_checkm_contamination,tree_mean_coverage,tree_shannon_entropy,tree_percent_max_uncovered_region,tree_largest_pileup,tree_largest_binned_pileup,tree_dist,tree_dist,tree_top_dist,tree_top_dist
0,2885.0,0.101421,0.442790,0.1609,0.2885,0.825389,0.096448,0.446408,0.012793,8.0,...,1.34,0.408550,1.154647,0.045304,13.0,16.0,0.005,0.005,2.0,2.0
1,31832.0,2.149773,1.910565,0.8376,3.1832,2.445698,0.883489,2.747820,0.003968,17.0,...,0.27,0.188667,0.636505,0.012494,15.0,19.0,0.006,0.006,1.0,1.0
2,38514.0,3.084938,2.319953,0.8852,3.8513,2.696329,0.954267,3.101891,0.005930,23.0,...,0.03,0.009710,0.048313,0.201170,11.0,9.0,0.042,0.042,1.0,1.0
3,69980.0,4.380529,3.045010,0.9607,6.9980,3.931844,0.987481,3.519081,0.010597,32.0,...,0.08,0.004080,0.021317,0.275227,15.0,16.0,0.019,0.019,2.0,2.0
4,39461.0,2.773138,2.416129,0.9197,3.9461,2.919896,0.937534,3.044385,0.001599,39.0,...,0.00,0.001083,0.007489,0.769399,5.0,3.0,0.059,0.059,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8761,1.0,0.000033,0.005706,0.0001,0.0001,0.009999,0.000033,0.000532,0.999967,1.0,...,0.00,0.000082,0.001238,0.561964,1.0,1.0,0.160,0.160,1.0,1.0
8762,1.0,0.000050,0.007071,0.0001,0.0001,0.009999,0.000050,0.000787,0.999950,1.0,...,0.00,0.000049,0.000771,0.999951,1.0,1.0,0.846,0.846,12.0,12.0
8763,1.0,0.000029,0.005418,0.0001,0.0001,0.009999,0.000029,0.000484,0.999971,1.0,...,0.19,0.026177,0.011693,0.144170,468.0,510.0,0.009,0.009,3.0,3.0
8764,1.0,0.000029,0.005401,0.0001,0.0001,0.009999,0.000029,0.000482,0.999971,1.0,...,0.80,0.000298,0.001899,0.496013,8.0,10.0,0.004,0.004,2.0,2.0


In [5]:
X = joblib.load("../data/X.pkl")
y = joblib.load("../data/y.pkl")

/home/bhillmann/.conda/envs/type_1/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
classifiers[0]

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('rfecv',
                 RFECV(cv=StratifiedKFold(n_splits=5, random_state=930525, shuffle=True),
                       estimator=LogisticRegression(class_weight='balanced',
                                                    penalty='l1',
                                                    solver='liblinear',
                                                    tol=0.001),
                       min_features_to_select=5, n_jobs=40, scoring='f1')),
                ('mlpclassifier',
                 MLPClassifier(alpha=0.01, learning_rate_init=0.01))])

In [7]:
shap.Explainer?

Object `shap.Explainer` not found.


/home/bhillmann/.conda/envs/type_1/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
import shap

# explain the model's predictions using SHAP
# (same syntax works for LightGBM, CatBoost, scikit-learn, transformers, Spark, etc.)
# explainer = shap.TreeExplainer(classifiers[0])
explainer = shap.KernelExplainer(classifiers[0],  data = X.head(50), link = "identity")

Provided model function fails when applied to the provided data set.


TypeError: 'Pipeline' object is not callable

In [ ]:
model = classifiers[0]

In [ ]:
pred = clf.predict_proba(X)[:, 0]

In [ ]:
# np.abs(shap_values.sum(1) + explainer.expected_value - pred).max()

In [ ]:
shap_values = explainer.shap_values(X)

In [ ]:
# explain the model's predictions using SHAP values
# (same syntax works for LightGBM, CatBoost, and scikit-learn models)

# visualize the first prediction's explanation
shap.plots.waterfall?

In [ ]:
shap.summary_plot(shap_values, X)

In [ ]:
shap.plots.beeswarm?

In [ ]:
shap.plots.beeswarm?

In [ ]:
shap_obj = explainer(X)

In [ ]:
# shap.plots.beeswarm(shap_obj)

In [ ]:
X['percent_coverage'].max()

In [ ]:
shap.dependence_plot("relative_abundance", shap_values[1], X, interaction_index = "percent_binned_coverage")

In [ ]:
shap.dependence_plot("tree_dist", shap_values[1], X, interaction_index = "relative_abundance")

In [ ]:
shap.dependence_plot("percent_binned_coverage", shap_values[1], X, interaction_index = "relative_abundance")

In [ ]:
# for name in X.columns:
#     shap.dependence_plot(name, shap_values[1], X)